# **Introduction to Model APIs in Keras**


In this lab, we will use a simple example to introduce you to different ways of creating models in Keras. 


## Objectives

After completing this lab you will be able to:

- __Understand__ different use cases for the Sequential and Functional APIs
- __Build__ custom models using sub-classing in Keras


## Setup


For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.
*   [`keras`](https://keras.io/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for loading datasets.


### Importing Required Libraries


In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)


2.12.0


## Types of Model APIs in Keras 


There are three main ways of creating models in Keras.


* **The Sequential Model**: This is a straightforward way of stacking layers but is limited to having a single-input and single-output stack of layers. 
* **The Functional API**: This can support arbitrary model architectures, and is more flexible in enabling complex implementations.

* **Model subclassing**: This is a way of implementing models from scratch. This is mainly used for out-of-the-box research use cases.


In the rest of the lab, we will go through all of these ways of creating models, and walk through a use case for implementing and training each of these model architectures in Keras and Tensorflow. 


### __Task Definition__


In this lab, we will be performing a simple classification task using the 'Sonar' dataset from UCI


> The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.



Let's start by reading in the dataset and defining our feature and target variables.


In [6]:
dataframe = pd.read_csv("sonar.csv", header=None)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# encode labels
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

In [7]:
dataframe

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [9]:
y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

In [8]:
encoded_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Now, we will split the dataset into a training and testing set with a 80:20 ratio.


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.20, random_state=42)

### **The Sequential Model API** 


The Sequential model API groups a linear stack of layers into a `tf.keras.Model`. It doesn't allow you to create models that share layers, have branches, multiple inputs, or multiple outputs. It does provide training and inference features on this model.  

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/sequential_model.png" alt="The Sequential model API" width="50%">



As you'll see, using the Sequential API is simple and straightforward, but is only appropriate for simpler, more straightforward tasks. Later in this notebook you'll spend some time building with a more flexible, powerful alternative: the Functional API.


### Create the Sequential Model


We start off by initializing the model as an instance of the Sequential class, like so: `model = Sequential()`. We follow this by adding layers, normalization layers, methods, softmax classifiers, and so on to the class, one at a time, using `model.add()`. This can be used to add different types of layer instances on top of the layer stack. Similarly, we can use `model.pop()` to remove the last layer of the model. 

Typically, the first layer receives an `input_shape` argument. This is because, in Keras, the shape of the weights is based on the shape of the inputs. The weights are only created when the model first sees some input data. After that, we can omit this argument and automate the inference of the input shape.

Note that if your model is non-linear or contains layers with multiple inputs or outputs, a Sequential model wouldn't be the right choice. 


In [13]:
def baseline_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [14]:
estimator = baseline_model()
estimator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                3660      
                                                                 
 dense_1 (Dense)             (None, 60)                3660      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


### Train and Evaluate the Model


After creating the model, compiling it with your choice of optimizer and loss function, and doing a sanity check on its contents, you are now ready to build.

Simply call the `.fit()` to train the model. 


In [15]:
# Write your solution here
estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 1s 4ms/step - loss: 0.6925 - accuracy: 0.5783
Epoch 2/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6645 - accuracy: 0.5904
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6546 - accuracy: 0.6205
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6370 - accuracy: 0.6506
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6204 - accuracy: 0.7349
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5981 - accuracy: 0.7771
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5836 - accuracy: 0.6807
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5624 - accuracy: 0.7711
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5483 - accuracy: 0.7169
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5218 - accuracy: 0.8253


After that completes, just use `.predict()` to evaluate against your test set. In this case, the `accuracy` will be used as the metric.


In [16]:
y_pred = estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 6ms/step


0.8571428571428571

### **The Functional Model API**


The Sequential class in Keras is typically used for implementing basic neural network architectures. The `Functional` class is typically preferred by most deep learning practitioners as it is more flexible and enables more complex implementations. It allows you to have non-linear topologies, different layers, multiple inputs and outputs. You can easily define branches in your architectures and easily share layers inside architectures. A Functional model is a graph where the nodes of the layers can connect in many more ways than one.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/functional_model.png" alt="The Sequential model API" width="50%">


All models that are implemented in `Sequential()` can easily be implemented in `Functional()`. Some examples of models that are implemented in `Functional()` include ResNet, GoogleNet, Xception. 


Both `Sequential()` and `Functional()` use the same training, evaluation, and inference functions. 

The Model class offers a built in training loop through the `.fit()` method, and a built-in evaluation loop using the `.evaluate()` method. 

The following is a simple example of fitting and evaluating a model on MNIST test data:    


We will start by loading the dataset directly using Keras.


In [17]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)
    
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [18]:
functional_estimator=functional_model()
estimator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                3660      
                                                                 
 dense_1 (Dense)             (None, 60)                3660      
                                                                 
 dense_2 (Dense)             (None, 1)                 61        
                                                                 
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


In [19]:
functional_estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 [==============================] - 1s 3ms/step - loss: 0.7001 - accuracy: 0.4880
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.6145
Epoch 3/10
11/11 [==============================] - 0s 2ms/step - loss: 0.6637 - accuracy: 0.6566
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6493 - accuracy: 0.7590
Epoch 5/10
11/11 [==============================] - 0s 2ms/step - loss: 0.6311 - accuracy: 0.7831
Epoch 6/10
11/11 [==============================] - 0s 2ms/step - loss: 0.6078 - accuracy: 0.7651
Epoch 7/10
11/11 [==============================] - 0s 2ms/step - loss: 0.5856 - accuracy: 0.7289
Epoch 8/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5616 - accuracy: 0.7952
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: 0.5376 - accuracy: 0.7711
Epoch 10/10
11/11 [==============================] - 0s 2ms/step - loss: 0.5115 - accuracy: 0.7892


Similar to what we did before, evaluate the estimator's performance on the test dataset, and print out the accuracy.


In [20]:
# Write your solution here
y_pred= functional_estimator.predict(X_test)
y_pred = [1 if x>=0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 [==============================] - 0s 5ms/step


0.9047619047619048

### **Model Subclassing**


Next, we will learn how to make new models and classes using model sub-classing. This method is more flexible and can be used to implement out-of-box models, but this comes at a cost, it is a lot harder to utilize than `Sequential()` and `Functional()` API.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/subclassing.png" alt="The Sequential model API" width="30%">


We will implement a custom model that performs a type of convolution or pooling. In the following example, we will implement a Sequential model but define it with multiple inputs and outputs. 


In [21]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(60, activation = 'relu')
        self.dense2 = Dense(60, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
def subclass_model():
    inputs = keras.Input(shape=(60,))
    mymodel = MyModel()
    outputs = mymodel.call(inputs)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

In [22]:
subclass_estimator = subclass_model()
subclass_estimator.fit(X_train, y_train, epochs=15, batch_size=16)

Epoch 1/15
11/11 [==============================] - 1s 3ms/step - loss: 0.6903 - accuracy: 0.5120
Epoch 2/15
11/11 [==============================] - 0s 2ms/step - loss: 0.6692 - accuracy: 0.6265
Epoch 3/15
11/11 [==============================] - 0s 2ms/step - loss: 0.6500 - accuracy: 0.7470
Epoch 4/15
11/11 [==============================] - 0s 2ms/step - loss: 0.6324 - accuracy: 0.7651
Epoch 5/15
11/11 [==============================] - 0s 3ms/step - loss: 0.6149 - accuracy: 0.7229
Epoch 6/15
11/11 [==============================] - 0s 2ms/step - loss: 0.5896 - accuracy: 0.8012
Epoch 7/15
11/11 [==============================] - 0s 2ms/step - loss: 0.5751 - accuracy: 0.7831
Epoch 8/15
11/11 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 0.8072
Epoch 9/15
11/11 [==============================] - 0s 2ms/step - loss: 0.5248 - accuracy: 0.7952
Epoch 10/15
11/11 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.7530
Epoch 11/15
11/11 [

In [23]:
y_pred = subclass_estimator.predict(X)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, encoded_y)

7/7 [==============================] - 0s 3ms/step


0.8509615384615384

## References


* Sequential Model: https://www.tensorflow.org/guide/keras/sequential_model

* Functional Model: https://www.tensorflow.org/guide/keras/functional
